# 内存管理

内存管理由 Python 解释器自动处理，开发者无需显式地进行内存分配和释放。


## 对象缓存优化

部分不可变类型对象在内存中缓存、重用。避免频繁创建/销毁对象，减少内存分配开销。


**小整数缓存**

`[-5, 257)` 区间的小整数会被缓存。


In [325]:
x = 256
y = 256
x is y

True

**字符串驻留**

由字母、数字、下划线组成的字符串会被缓存。


In [326]:
a = "hello_world_001"
b = "hello_world_001"
a is b

True

**`ASCII` 单字符驻留**


In [327]:
a = "~"
b = "~"
a is b

True

> `sys.intern` 方法可手动创建驻留字符串。


In [328]:
import sys

a = sys.intern("你好")
b = sys.intern("你好")
a is b

True

**空元组缓存**


In [329]:
a = ()
b = ()
a is b

True

## 垃圾回收


### 回收算法


**引用计数**

引用计数通过为每个对象添加一个计数器来记录该对象被引用的次数。每当有一个新的引用指向该对象时，计数器就会增加 1；每当有一个引用不再指向该对象时，计数器就会减少 1。当计数器的值降为 0 时，就表示没有任何引用再指向该对象，因此该对象不再被使用，可以被回收。引用计数是实时进行的。

> 两个或多个对象之间相互引用形成一个环即为循环引用，引用计数机制无法处理对象循环引用。


> `sys.getrefcount` 方法可获取对象引用计数。注意 `sys.getrefcount` 方法本身会增加一个对象临时引用。


In [330]:
from random import random
import sys

x = random()  # 生成一个没有被内存缓存的随机数

sys.getrefcount(x)  # 获取对象引用计数

2

In [331]:
sys.getsizeof(x)  # 获取对象占用内存字节数

24

In [332]:
ref = x

sys.getrefcount(x)

3

In [333]:
del ref  # 删除一次引用

sys.getrefcount(x)

2

**标记清除**

检测和回收不再被使用的对象。可处理循环引用。

- 标记阶段：垃圾回收器从根对象（全局变量和活动栈中的局部变量等）递归遍历对象图，标记所有可达对象。
- 清除阶段：遍历所有对象并回收垃圾对象（未标记对象）。

> 可达对象：在程序中能够被访问到的对象，也就是与根对象之间存在引用链的对象。

> 活动栈：包含函数调用上下文信息（局部变量、参数等）的栈。


In [334]:
# 变量 a，b 引用的对象即为相互引用的循环引用对象
a = {}
b = {"a": a}
a["b"] = b

# 释放变量 a，b，此时两对象为不可达状态
del a, b

### 分代回收策略

Python 将所有对象分为三个“代”，即 0 代、1 代和 2 代。新创建的对象被放入 0 代，而那些在垃圾回收中存活下来的对象会被提升到更高的代。这样，频繁创建和销毁的短命对象主要集中在 0 代，而长寿命的对象则逐渐被提升到高代。

每个代都有一个阈值，当该代的对象数量达到阈值时，就会触发该代的垃圾回收。在进行高代的垃圾回收时，也会同时对低代进行垃圾回收。由于 0 代主要包含短命对象，所以 0 代的垃圾回收最为频繁，这有助于快速回收那些短命的临时对象。而高代的垃圾回收则不那么频繁，但每次回收会涉及更多的对象，这有助于回收那些长寿命的，可能存在循环引用的对象。


> `gc.get_threshold` 方法可获取每代的回收阈值


In [335]:
import gc

gc.get_threshold()

(700, 10, 10)

In [336]:
# 自上一次垃圾回收以来，每一代新增的对象数量
gc.get_count()

(438, 0, 0)

每个代别垃圾回收的统计信息。

- collections：此代已进行的垃圾收集次数。
- collected：此代已收集的对象总数。
- uncollectable：此代发现但无法收集的对象总数。


In [337]:
gc.get_stats()

[{'collections': 247, 'collected': 1604, 'uncollectable': 0},
 {'collections': 22, 'collected': 833, 'uncollectable': 0},
 {'collections': 7, 'collected': 743, 'uncollectable': 0}]

[Garbage collection in Python: things you need to know](https://rushter.com/blog/python-garbage-collector/)

[The Garbage Collector](https://pythoninternal.wordpress.com/2014/08/04/the-garbage-collector/)
